In [1]:
import pandas as pd
from itertools import chain
import numpy as np

In [ ]:
import pip
pip.main(["install", "openpyxl"])

In [2]:
meses = [
'ENERO',
'FEBRERO',
'MARZO',
'ABRIL',
 'MAYO',
 'JUNIO',
 'JULIO',
 'AGOSTO',
 'SEPTIEMBRE',
 'OCTUBRE',
 'NOVIEMBRE',
 'DICIEMBRE']
rangosHora = None
year = '2015'

Importación de datos de excel desde 2013 - 2019

In [3]:
def convertirAFormato24h(texto):
    if 'am' not in texto and 'pm' not in texto:
        return texto
    texto = texto.replace("-","")
    textoPartido = texto.split()
    if 'am'  in textoPartido:
        return textoPartido[0] + ' - ' + textoPartido[1]
    else:
        hDes = int(textoPartido[0])
        hHas = int(textoPartido[1])
        hHas = hHas + 12
        hDes = hDes + 12 if hHas > 14 else hDes
        return str(hDes) + ' - ' + str(hHas)

In [4]:
def tratarHoja (hoja):   
    global rangosHora 
    hoja = hoja.iloc[1:, 1:len(hoja.columns)-1]
    hoja = hoja.T
    hoja.columns = hoja.iloc[0]
    colTotal = list(hoja.columns).index('TOTAL')
    hoja = hoja.iloc[1:,0:colTotal]
    rangosHora = [convertirAFormato24h(s) for s in hoja.columns[1:7]]
    hoja.columns = ['Dia'] + rangosHora
    return hoja

In [5]:
def resumirHoja(hoja,mes,year):
    global rangosHora
    hoja['Personas'] = [tuple(x) for x in (hoja[rangosHora]).values]
    hoja['Dia'] = hoja['Dia'].astype(int).astype(str)
    lens = list(map(len, hoja.Personas))
    res = pd.DataFrame({'Dia': np.repeat(hoja.Dia,lens),
                        'Mes':np.tile([mes],len(lens)*len(rangosHora)),
                        'Año':np.tile([year],len(lens)*len(rangosHora)),
                        'RangoHora': np.tile(rangosHora,len(lens)),
                        'Aforo': list(chain.from_iterable(hoja.Personas))})
    return res

In [6]:
def iterarAnio(year):    
    xlsx = pd.ExcelFile('./Aforo Biblioteca Proyecto Maestría/Ingreso a edificio '+year+'.xlsx', engine='openpyxl')
    hojas = [x.upper() for x in xlsx.sheet_names if x in meses]
    pdMesesResumidos = []
    for mes in hojas:
        hoja = pd.read_excel(xlsx, mes)
        hoja = tratarHoja(hoja)
        pdMesesResumidos.append(resumirHoja(hoja,mes, year))
    return pd.concat(pdMesesResumidos)

In [7]:
anios = list(range(2013,2019))
pdList = []
for anio in anios:
    pdList.append(iterarAnio(str(anio)))
resumenCompleto = pd.concat(pdList, ignore_index=True)
print(resumenCompleto)

      Dia        Mes   Año RangoHora  Aforo
0       1      ABRIL  2013     0 - 6   34.0
1       1      ABRIL  2013     6 - 9  360.0
2       1      ABRIL  2013    9 - 14  267.0
3       1      ABRIL  2013   14 - 18  106.0
4       1      ABRIL  2013   18 - 24    0.0
...    ..        ...   ...       ...    ...
10210  14  DICIEMBRE  2018     6 - 9  115.0
10211  14  DICIEMBRE  2018    9 - 14  311.0
10212  14  DICIEMBRE  2018   14 - 18   70.0
10213  14  DICIEMBRE  2018   18 - 20    0.0
10214  14  DICIEMBRE  2018   20 - 24    1.0

[10215 rows x 5 columns]
